# Bulk-apply confidence intervals calculations to results
Apply the calculation of confidence intervals through the `confidence_intervals` package to all model results in respective output folders. This is done by automatically loading the predictions DataFrames and feeding them into the package. The results are saves as aggregated .csv files. Optionally, in the last cell, the column `lower_bound_larger_null` is added to check if the lower bound of the confidence interval is larger than 0.

In [1]:
import os
import pandas as pd
import audbenchmark
from tqdm import tqdm

# Import bootstrapping
from confidence_intervals import evaluate_with_conf_int

1) Load the compiled results DataFrames
2) Calculate confidence intervals on all "Path" given
   1) Sort the results acc. to the best declared performance
   2) Maybe also highlight models with significant lower bounds
3) Save the results

In [2]:
# Add a meaningful term to summarise the conditions compiled
file_out_composed = "compiled-outer_cv_loso"

lst_conditions = [
    "outer_cv_loso-surveys_all",
    "outer_cv_loso-survey_daily",
    "outer_cv_loso-surveys_weekly",
    #
    # "outer_cv_loso-noisy-surveys_all",
    # "outer_cv_loso-noisy-surveys_daily",
    # "outer_cv_loso-noisy-surveys_weekly",
    # "outer_cv_loso-noisy-surveys_weekly-stress_current_raw",
]
path_composed = "../../results/mwas/composed"
path_results_base = "/data/phecker/project/audiary/projects/prompt-performance/modelling-static/results/mwas/modelling"
metric = audbenchmark.metric.concordance_cc

In [3]:
# Initialize a list to store the rows with the highest ccc_conf_mean
meta_data = []

for cur_condition in tqdm(lst_conditions, desc="Conditions"):
    path_condition = os.path.join(path_composed, cur_condition)

    # Get list of all .csv files in path_condition
    csv_files = [f for f in os.listdir(path_condition) if f.endswith(".csv")]

    # Iterate over all .csv files in path_condition with a progress bar
    for file_name in tqdm(csv_files, desc=f"Targets {cur_condition}", leave=False):
        if file_name.endswith(".csv"):
            # Construct the full file path
            file_path = os.path.join(path_condition, file_name)

            # Load the .csv file
            df_results = pd.read_csv(file_path)

            # Extract the target from the file name
            target = file_name.replace("results-", "").replace(".csv", "")

            # Initialize lists to store the results
            ccc_conf_mean = []
            ccc_conf_low = []
            ccc_conf_high = []
            lower_bound_larger_null = []
            tasks = []
            features = []

            # Iterate over every row in the column "path"
            for str_path in tqdm(
                df_results["path"],
                desc=f"individual results {cur_condition}",
                leave=False,
            ):
                # Remove the leading forward slash
                str_path = str_path.lstrip("/")

                # Extract the task and features
                path_parts = str_path.split("/")
                task = path_parts[2] if len(path_parts) > 2 else ""
                feature = path_parts[7] if len(path_parts) > 7 else ""

                # Remove the trailing part starting from "models/results-compiled.yaml"
                str_path = str_path.split("models/results-compiled.yaml")[0]

                # Construct the path to the parquet file
                path = os.path.join(
                    path_results_base, str_path, "data/df_results_test.parquet.zstd"
                )

                # Load the parquet file
                df_test = pd.read_parquet(path)

                # Evaluate with confidence intervals
                tpl_result = evaluate_with_conf_int(
                    samples=df_test["predictions"].values,
                    metric=metric,
                    labels=df_test[target].values,
                    conditions=df_test["participant_code"].values,
                    num_bootstraps=1000,
                    alpha=5,
                )

                # Extract the mean and confidence interval values
                mean_value = tpl_result[0]
                conf_int_low = tpl_result[1][0]
                conf_int_high = tpl_result[1][1]

                # Append the results to the lists
                ccc_conf_mean.append(mean_value)
                ccc_conf_low.append(conf_int_low)
                ccc_conf_high.append(conf_int_high)
                lower_bound_larger_null.append(conf_int_low > 0)
                tasks.append(task)
                features.append(feature)

            # Add the results as new columns to df_results at the beginning
            df_results.insert(0, "ccc_conf_mean", ccc_conf_mean)
            df_results.insert(1, "ccc_conf_low", ccc_conf_low)
            df_results.insert(2, "ccc_conf_high", ccc_conf_high)
            df_results.insert(3, "lower_bound_larger_null", lower_bound_larger_null)
            df_results.insert(4, "task", tasks)
            df_results.insert(5, "features", features)

            # Sort df_results by ccc_conf_mean
            df_results = df_results.sort_values(by="ccc_conf_mean", ascending=False)

            # Find the row with the highest ccc_conf_mean
            best_row = df_results.iloc[0].copy()
            best_row["survey"] = cur_condition
            best_row["target"] = target

            # Append the best row to the meta_data list
            meta_data.append(best_row)

            # Construct the new file path with "-conf" appended
            new_file_path = file_path.replace(".csv", "-conf.csv")

            # Write the updated DataFrame back to the new file path
            df_results.to_csv(new_file_path, index=False)

# Convert the meta_data list to a DataFrame
df_meta = pd.DataFrame(meta_data)

# Ensure the columns "survey" and "target" are at the beginning
columns = ["survey", "target"] + [
    col for col in df_meta.columns if col not in ["survey", "target"]
]
df_meta = df_meta[columns]

# Save the meta DataFrame to path_composed as "compiled.csv"
compiled_path = os.path.join(path_composed, file_out_composed + ".csv")
df_meta.to_csv(compiled_path, index=False)

Conditions:   0%|          | 0/1 [00:00<?, ?it/s]

















































































































































































































































































































































































Conditions: 100%|██████████| 1/1 [59:40<00:00, 3580.46s/it]


In [4]:
# Initialize a list to store the rows with the highest ccc_conf_mean
meta_data = []

for cur_condition in tqdm(lst_conditions, desc="Conditions"):
    path_condition = os.path.join(path_composed, cur_condition)

    # Get list of all .csv files that are already processed ("-conf") in path_condition
    csv_files = [f for f in os.listdir(path_condition) if f.endswith("-conf.csv")]

    # Iterate over all .csv files in path_condition with a progress bar
    for file_name in tqdm(csv_files, desc=f"Targets {cur_condition}", leave=False):
        # Construct the full file path
        file_path = os.path.join(path_condition, file_name)

        # Load the .csv file
        df_results = pd.read_csv(file_path)

        # Extract the target from the file name
        target = file_name.replace("results-", "").replace("-conf.csv", "")

        # Find the first row where "lower_bound_larger_null" is True
        matching_row = df_results[df_results["lower_bound_larger_null"] == True].head(1)

        if not matching_row.empty:
            # Add cur_condition and target to the matching row
            matching_row.insert(0, "survey", cur_condition)
            matching_row.insert(1, "target", target)
            # Append the matching row to meta_data
            meta_data.append(matching_row.iloc[0])
        else:
            # Append an empty row with cur_condition and target
            empty_row = pd.Series({"survey": cur_condition, "target": target})
            meta_data.append(empty_row)

# Convert the meta_data list to a DataFrame
df_meta = pd.DataFrame(meta_data)

# Ensure the columns "survey" and "target" are at the beginning
columns = ["survey", "target"] + [
    col for col in df_meta.columns if col not in ["survey", "target"]
]
df_meta = df_meta[columns]

# Save the meta DataFrame to path_composed as "compiled.csv"
compiled_path = os.path.join(path_composed, file_out_composed + "-significant.csv")
df_meta.to_csv(compiled_path, index=False)

Conditions: 100%|██████████| 1/1 [00:00<00:00, 26.16it/s]
